In [ ]:
# Importing 
import requests
import csv 
import re
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from google.colab import files
import datetime
from datetime import date
from pytz import timezone
eastern = timezone('US/Eastern')

In [ ]:
# get Event data page 

events_upcoming  = 'http://ufcstats.com/statistics/events/upcoming?page=all'
events_completed = 'http://ufcstats.com/statistics/events/completed?page=all'

# get all event details from both those pages and add to a big list 
# loop through the big list and retrieve relavent information (date, location, fightsListed)

event_string = 'http://ufcstats.com/event-details/'

completed_page       = requests.get(events_completed)
completed_soup       = BeautifulSoup(completed_page.text, 'html.parser')
all_completed_events = completed_soup.find_all('a')

upcoming_page        = requests.get(events_upcoming)
upcoming_soup        = BeautifulSoup(upcoming_page.text, 'html.parser')
all_upcoming_events  = upcoming_soup.find_all('a')

ufcEventsLinks = []
ufcEvents      = []

# completed events page list 
for event in all_completed_events:
  link = str(event.get('href'))
  if event_string in link: 
    ufcEventsLinks.append(link) 
    print(link)
print('----------------------------------')
# upcoming events page links 
for event in all_upcoming_events:
  link = str(event.get('href'))
  if event_string in link:
    ufcEventsLinks.append(link)
    print(link)


In [ ]:
ufcEventsLinks

In [ ]:
for eventLink in ufcEventsLinks:
  eventRow = []
  title           = "---"
  location        = "---"
  date            = "---"
  listedFights    = 0 
  
  event_page = requests.get(eventLink)
  event_soup = BeautifulSoup(event_page.text, 'html.parser')
  event_details = event_soup.find_all('li', {'class' : 'b-list__box-list-item'})
  event_title   = event_soup.find_all('h2', {'class' : 'b-content__title' })
  event_fights  = event_soup.find_all('a')

  for text in event_title:
    title = text.text.strip()

  for detail in event_details:
    detail_text = detail.text.strip()
    if 'Date:' in detail_text: 
          date = detail_text.replace('Date:', '').strip()
    if 'Location' in detail_text: 
          location = detail_text.replace('Location:', '').strip()

  # count fights in every completed event 
  fight_string = 'http://ufcstats.com/fight-details/'
  for fight in event_fights:
    if fight_string in str(fight.get('href')):
        listedFights += 1

  # print(eventLink)
  # print(title)
  # print(date)
  # print(location)
  # print(listedFights)    
 
  eventRow.append(eventLink)
  eventRow.append(title)
  eventRow.append(date)
  eventRow.append(location)
  eventRow.append(listedFights)
  ufcEvents.append(eventRow)

  # Code takes 4 minutes to run 
  

In [ ]:
# Create and clean dataset 

today = datetime.date.today()
print("Today's date:", today)

df = pd.DataFrame(ufcEvents, columns=['eventLink', 'eventName', 'eventDateString', 'eventLocation', \
                                     'eventListedFights'])


# Clean/engineer dataset 
df['eventDateString'] = df['eventDateString'].replace(',', '', regex=True)
df['date'] = pd.to_datetime(df['eventDateString'], format="%B %d %Y")
df['date'] = df['date'].dt.date
df = df.sort_values(by=['date'], ascending=False)
df['upcoming'] = np.where(df['date']>today,1,0)
df

# df['date'] = df['eventDateString'].datetime.strptime(df['eventDateString'], "%B %d %Y")



In [ ]:
# Export to Drive and Download
df.to_excel('/content/drive/MyDrive/UFC Model/ufcEventsList.xlsx')
files.download('/content/drive/MyDrive/UFC Model/ufcEventsList.xlsx')


In [ ]:
  event_page = requests.get('http://ufcstats.com/event-details/509697e08673d2e5')
  event_soup = BeautifulSoup(event_page.text, 'html.parser')
  event_details = event_soup.find_all('li', {'class' : 'b-list__box-list-item'})
  event_title   = event_soup.find_all('h2', {'class' : 'b-content__title' })
  event_fights  = event_soup.find_all('a')

In [ ]:
  listedFights = 0
  fight_string = "http://ufcstats.com/fight-details/"
  
  for fight in event_fights:
    print(str(fight.get('href')))
    if fight_string in str(fight.get('href')):
        listedFights += 1
        print(str(fight.get('href')))
  listedFights      

In [ ]:
event_fights